# you.com News API <> Llama Index

Creating a chatbot that can provide personalized news on any topic using you.com news API involves several steps, including obtaining the necessary credentials, and programming your chatbot to use the API. 

## Install required packages

In [ ]:
%%capture
! pip install llama-index-retrievers-you==0.1.2
! pip install dotenv==0.0.5

## Load API keys

In [7]:
# assumes a .env file exists with api keys YDC_API_KEY and OPENAI_API_KEY

from dotenv import load_dotenv
import os

load_dotenv()

True

## Create a retriever

In [3]:
# TODO update the pinned version of llama-index-retrievers-you after PR is merged: https://github.com/run-llama/llama_index/pull/13934
from llama_index.retrievers.you import YouRetriever

retriever = YouRetriever(endpoint_type="news")
retrieved_results = retriever.retrieve("national parks in the US")

print(retrieved_results[0].get_content())

Our 63 US National Parks across America are a collection of some of the most pristine and beautifully preserved landscapes in the world. From sea to shining sea, nationally protected treasures are filled with stunning views, remarkable wildlife, and historic landmarks.
URL: https://brightstandards.com/us-national-parks-list-map/
age: 5 days ago


# Create a chat engine

Conceptually, it is a stateful analogy of a Query Engine. 

In [4]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Canada")

print(response)

The recent news articles provide a snapshot of various events and developments in Canada. Here are some key highlights:

1. The Bank of Canada cut its key policy rate, becoming the first G7 country to do so in four years. Economists predict further cuts in July.
   Source: Reuters, Yahoo Finance, CNBC

2. Canada Border Services Agency workers are set to strike, potentially causing border chaos.
   Source: CBC News, WCAX

3. Canada demands a portion of revenue from streaming services to support local news and content.
   Source: Ars Technica, Reuters

4. Wall Street anticipates a weaker Canadian dollar and more interest rate cuts following the Bank of Canada's move.
   Source: Bloomberg

5. There are calls to end the practice of "flagpoling" in Canada.
   Source: WCAX

6. Canadian officials are preparing for a meeting with French President Macron, US President Biden, UK's King Charles III, and Canada's PM Trudeau to commemorate D-Day.
   Source: The Guardian

7. Online streaming service

### Include a timeframe

In [5]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Norway in Jan 2024")

print(response)

In January 2024, Norway experienced some significant events and developments:

1. Norway's $1.6 trillion sovereign wealth fund, the world's largest, reported a record profit of 2.22 trillion crowns ($213 billion) in 2023, driven by strong returns on its investments in technology stocks.

2. January saw combined electric vehicles (EVs) at a 93.9% share in Norway, with a record 92.1% being full electrics (BEVs) and 1.8% plug-in hybrids (PHEVs). This marked a significant increase compared to the previous year.

3. Norway experienced a market crash of -76.6% in January 2023, leading to the lowest market performance in 61 years due to price increases.

4. The country saw a surge in new electric car registrations, with 4,717 new electric cars and 94 plug-in hybrids registered in January 2024.

5. Norway's internet penetration stood at 99.0%, with 5.44 million internet users and 4.49 million social media users in January 2024.

These events showcase Norway's strong focus on sustainability, te

# Create a chat engine with memory

By keeping track of the conversation history, it can answer questions with past context in mind.
 
`condense_plus_context` - A combination of condense_question and context. Look at the chat history and re-write the user message to be a retrieval query for the index. The retrieved text is inserted into the system prompt, so that the chat engine can either respond naturally or use the context from the query engine.

In [63]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.llms.openai import OpenAI

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
llm = OpenAI(model="gpt-4o")

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    memory=memory,
    llm=llm,
)

In [64]:
# relevant news article with a summary of the news
response = chat_engine.chat("Salesforce")
print(response)

Salesforce has recently been in the spotlight due to a significant drop in its stock price. The company reported its fiscal first-quarter (FY 2025) earnings, which missed revenue consensus estimates, leading to a sharp decline of more than 20% in one day. This was the largest one-day loss for Salesforce in nearly 20 years. The disappointing earnings report has raised concerns among investors about the adoption curve and ultimate monetization of generative AI for software-as-a-service firms like Salesforce.

Additionally, Salesforce's CEO Marc Benioff mentioned that the company is remaining cautious around mergers and acquisitions (M&A) but is open to deals that are seen as additive. Despite this cautious stance, the weak outlook has further worried investors.

The stock's poor performance has also placed it among the worst large-cap performers recently, alongside companies like MongoDB, Nutanix, and DraftKings. Some investors, like Bret Jensen, are looking to take advantage of the sell

In [65]:
# uses memory to relate the Hubspot query to market performance news in the query above
response = chat_engine.chat("Hubspot")
print(response)

HubSpot has been making headlines recently for several reasons:

1. **Potential Acquisition by Alphabet (Google)**:
   - There have been multiple reports confirming that Alphabet, Google's parent company, is in talks to acquire HubSpot. This potential acquisition is expected to be an all-stock deal. The news has caused HubSpot's shares to jump significantly, with a notable 9.5% increase in the morning session following the confirmation of these talks by CNBC's David Faber. This potential acquisition has been widely reported across various financial news platforms, including CNBC, Yahoo Finance, The Motley Fool, Benzinga, and Seeking Alpha.

2. **New Features and Compliance Tools**:
   - HubSpot has recently announced new tools to help businesses manage confidential customer data within its Smart CRM. This includes support for HIPAA (Health Insurance Portability and Accountability Act) compliance, which is crucial for businesses handling sensitive health information. This update is aime

## Personalize news

Using a custom prompt, this news feed can be customized to specific threads, such as this one focused on politics and economics.

In [61]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core import PromptTemplate

custom_prompt = PromptTemplate(
    """\
You're a top-tier reporter focused on news about {question}.\n
YOU SHOULD ALWAYS RESEARCH THE FOLLOWING QUERIES: \n
1. {question} recent news and latest development\n
2. recent regulatory changes affecting {question}\n
3. recent economic events impacting {question}\n
"""
)

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
llm = OpenAI(model="gpt-4o")

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    condense_question_prompt=custom_prompt,
    memory=memory,
    llm=llm,
)

In [62]:
response = chat_engine.chat("Mongolia")
print(response)

Mongolia is currently involved in several significant activities and developments:

1. **Diplomatic Efforts and International Relations**:
   - **Japan**: Japanese Prime Minister Fumio Kishida is considering visiting Mongolia in August to seek support for advancing stalled talks with North Korea regarding the abduction of Japanese nationals.
   - **Ukraine**: Ukraine has called on Mongolia to participate in the Global Peace Summit in Switzerland.
   - **Belarus**: Mongolia and Belarus have signed a number of documents to further develop bilateral relations and cooperation. Belarusian President Alexander Lukashenko recently held talks with his Mongolian counterpart during a visit to Ulaanbaatar.

2. **Economic and Technological Developments**:
   - **Uranium Market**: Mongolia is emerging as a hub for the uranium market, balancing interests between the East and West.
   - **Digital Payments**: Users of 12 popular overseas e-wallets from 8 countries and regions can now enjoy seamless dig